In [90]:
# Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from flask import Flask, render_template, redirect
import requests
import pymongo
import pandas as pd


In [91]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/100.0.4896.60/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\radgu\.wdm\drivers\chromedriver\win32\100.0.4896.60]


In [92]:
# URL of the page to be scrapped
url = 'https://redplanetscience.com/'
browser.visit(url)

In [93]:
### NASA Mars News

# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Retrieve the latest news title
news_title=soup.find_all('div', class_='content_title')[0].text
news_p=soup.find_all('div', class_='article_teaser_body')[0].text

news_title
news_p

"The clean room at NASA's Jet Propulsion Laboratory was open to the media to see NASA's next Mars explorer before it leaves for Florida in preparation for a summertime launch."

In [94]:
### JPL Mars Space Images - Featured Image
jpl_url="https://spaceimages-mars.com"
browser.visit(jpl_url)

# HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

image_block=soup.find('img', class_='headerimage fade-in')

image_url = image_block.get("src")

featured_image_url=jpl_url+ "/" + image_url

featured_image_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

In [95]:
### Mars Facts

#Visit the Mars Facts webpage [here](https://galaxyfacts-mars.com) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
#Use Pandas to convert the data to a HTML table string.

url='https://galaxyfacts-mars.com'
tables=pd.read_html(url)
type(tables)
df = tables[0]
df.head()


,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"


In [96]:
#drop single header rows


df.head()
mars_fact=df.rename(columns={0:"Stat", 1:"Value",2:"dropme"},errors="raise")

#df.drop('dropme',axis=1,inplace=True)
#mars_fact.set_index("Stat",inplace=True)

mars_fact.columns = mars_fact.columns.get_level_values(0)

# remove column 
mars_fact = mars_fact.drop(mars_fact.columns[2], axis=1)
#drop first row
mars_fact = mars_fact.iloc[1:,:]

mars_fact = mars_fact.set_index('Stat')

mars_fact



,Value
Stat,
Diameter:,"6,779 km"
Mass:,6.39 × 10^23 kg
Moons:,2
Distance from Sun:,"227,943,824 km"
Length of Year:,687 Earth days
Temperature:,-87 to -5 °C


In [97]:
#Use to_html method to generate HTML tables from df
html_table = mars_fact.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Stat</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n  </tbody>\n</table>'

In [98]:
#Save as html file
mars_fact.to_html('mars_table.html')

In [105]:
### Mars Hemispheres

# Setting url for alternate browser
url = 'https://marshemispheres.com/'
browser.visit(url)
html=browser.html
soup=BeautifulSoup(html,'html.parser')

In [106]:
# Extract hemispheres item elements
mars_hems=soup.find('div',class_='collapsible results')
mars_item=mars_hems.find_all('div',class_='item')
hemisphere_image_urls=[]


In [107]:
# Loop through each hemisphere item
for item in mars_item:
    # Error handling
    try:
        # Extract title
        hem=item.find('div',class_='description')
        title=hem.h3.text
        # Extract image url
        hem_url=hem.a['href']
        browser.visit(url+hem_url)
        html=browser.html
        soup=BeautifulSoup(html,'html.parser')
        image_src=soup.find('li').a['href']
        image_src=url + image_src
        if (title and image_src):
            # Print results
            print('-'*50)
            print(title)
            print(image_src)
        # Create dictionary for title and url
        hem_dict={
            'title':title,
            'image_url':image_src
        }
        hemisphere_image_urls.append(hem_dict)
    except Exception as e:
        print(e)

--------------------------------------------------
Cerberus Hemisphere Enhanced
https://marshemispheres.com/images/full.jpg
--------------------------------------------------
Schiaparelli Hemisphere Enhanced
https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg
--------------------------------------------------
Syrtis Major Hemisphere Enhanced
https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg
--------------------------------------------------
Valles Marineris Hemisphere Enhanced
https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg


In [108]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]